In [1]:
import numpy as np
import pandas as pd

from pyspark.context import SparkContext
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import SparkSession

In [2]:
# In Maman 22, our goal is to analyze the wine dataset from Maman 21.
# Since we already cleaned it on the previous project, we can reuse it.

data = pd.read_csv('winequality-white-clean.csv').drop(columns = ['Unnamed: 0'])
data.head()

,FixedAcidity,VolatileAcidity,CitricAcid,ResidualSugar,Chlorides,FreeSulfurDioxide,TotalSulfurDioxide,Density,pH,Sulphates,Alcohol,Quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
pd.plotting.scatter_matrix(data, alpha = 0.3, figsize = (16,16), diagonal = 'kde');

In [4]:
# As we can see, all the features besides Quality are continuous,
# we must bin them in order to use Association Rules algorithms.
# We will bin the Quality feature as well because it is not nicely distributed
# the dataset will be transformed into quantiles for each feature.

# VolatileAcidity, ResidualSugar, Sulphates, Alcohol will divide into 4 quantiles,
# as their distribution is not mean centered, the other features will divide into 3 quantiles.
# Quality will devide same as maman 21

for c in ['VolatileAcidity', 'ResidualSugar', 'Sulphates', 'Alcohol']:
    data[c + '_bins'] = pd.qcut(data[c], 4, labels=["Low", "LowMid", "MidHigh", "High"])

for c in ['FixedAcidity', 'CitricAcid', 'Chlorides', 'FreeSulfurDioxide', 'TotalSulfurDioxide', 'Density', 'pH']:
    data[c + '_bins'] = pd.qcut(data[c], 3, labels=["Low", "Mid", "High"])

data['Quality_bins'] = pd.cut(data['Quality'], [0,5,6,10], labels=["Bad", "Mid", "Good"])
    
data = data.iloc[:,12:]
data.head()

,VolatileAcidity_bins,ResidualSugar_bins,Sulphates_bins,Alcohol_bins,FixedAcidity_bins,CitricAcid_bins,Chlorides_bins,FreeSulfurDioxide_bins,TotalSulfurDioxide_bins,Density_bins,pH_bins,Quality_bins
0,MidHigh,High,LowMid,Low,Mid,High,Mid,High,High,High,Low,Mid
1,MidHigh,Low,MidHigh,Low,Low,Mid,High,Low,Mid,Mid,High,Mid
2,MidHigh,MidHigh,LowMid,LowMid,High,High,High,Mid,Low,Mid,High,Mid
3,LowMid,MidHigh,Low,LowMid,High,Mid,High,High,High,High,Mid,Mid
4,LowMid,MidHigh,Low,LowMid,High,Mid,High,High,High,High,Mid,Mid


In [5]:
# start Spark session
# prepare data to spack convention and create spark DataFrame

sc = SparkContext('local')
spark = SparkSession(sc)

data_dummies = pd.get_dummies(data)
data_spark_ready = [(i,data_dummies.columns[data_dummies.iloc[i].values==1].to_list()) for i in data.index]


df = spark.createDataFrame(data_spark_ready, ["id", "items"])

# PF-Growth

The first model selected is FP Growth. 
It is an efficient and scalable method for mining the complete set of frequent patterns by pattern fragment growth.
It is implemented by a data structure called FP-Tree, which represent frequent items.

FP-tree construction:

    Input: A transaction database DB and a minimum support threshold.
    
    Output: FP-tree, the frequent-pattern tree of DB.
    
    Method: The FP-tree is constructed as follows.
        Scan the transaction database DB once. 
        Collect F, the set of frequent items, and the support of each frequent item. 
        Sort F in support-descending order as FList, the list of frequent items.
        Create the root of an FP-tree, T, and label it as “null”.
        For each transaction Trans in DB do the following:
            * Select the frequent items in Trans and sort them according to the order of FList.
              Let the sorted frequent-item list in Trans be [ p | P], where p is the first element and P is the  remaining list. 
              Call insert tree([ p | P], T ).
            * The function insert tree([ p | P], T ) is performed as follows. 
              If T has a child N such that N.item-name = p.item-name, then increment N ’s count by 1; else create a new node N, with its count initialized to 1, its parent link linked to T, and its node-link linked to the nodes with the same item-name via the node-link structure. If P is nonempty, call insert tree(P, N ) recursively.

After the construction, the tree can be mined for frequent items
From the frequent items, we proceed to a search of meaningful once ( Like we will do with Lift value )

The algorithm will provide us with a lot of rules, but some of them can be obvious or not meaningful,
I order to reduce them, we will increase the support level, which controls how much each unique item relatively appear in the dataset.

In [6]:
# Build FPGrowth tree
# Remark: the project demand for min Support 0.4 brings no results, 
# so lower support was selected in order to continue with the exercise


fpGrowth = FPGrowth(itemsCol="items", minSupport=0.1, minConfidence=0.6)
model = fpGrowth.fit(df)

In [7]:
# Get Most interesting rules:
# In order to pick the most interesting rules, the Lift index was selected.
# Lift(A -> B) refers to the increase in the ratio of B to appeare when A is present.
# Lift(A -> B) can be calculated by dividing Confidence(A -> B) divided by Support(B).
# Mathematically it can be represented as:
# Lift(A→B) = (Confidence (A→B))/(Support (B))
# Lower lift will result in more rules that have less significant. 

FP_rules = model.associationRules.toPandas()
FP_rules[FP_rules.lift>2.5].sort_values(by=['lift'], ascending=False)  

# 36 rules of all the rules found can considered interesting and meaningfool

,antecedent,consequent,confidence,lift
8,"[Density_bins_High, pH_bins_Low]",[ResidualSugar_bins_High],0.816038,3.286413
14,"[ResidualSugar_bins_LowMid, Density_bins_Low]",[Alcohol_bins_High],0.712644,3.011206
80,"[ResidualSugar_bins_High, Alcohol_bins_Low, To...",[Density_bins_High],1.000000,3.003179
64,"[ResidualSugar_bins_High, Alcohol_bins_Low]",[Density_bins_High],1.000000,3.003179
21,"[Density_bins_Low, Chlorides_bins_Low]",[Alcohol_bins_High],0.699029,2.953679
40,"[ResidualSugar_bins_High, TotalSulfurDioxide_b...",[Density_bins_High],0.978947,2.939954
26,"[ResidualSugar_bins_High, Chlorides_bins_High]",[Density_bins_High],0.971223,2.916756
53,"[Alcohol_bins_Low, Density_bins_High]",[ResidualSugar_bins_High],0.707340,2.848658
19,"[FreeSulfurDioxide_bins_High, Density_bins_High]",[ResidualSugar_bins_High],0.705536,2.841391
46,"[ResidualSugar_bins_High, FreeSulfurDioxide_bi...",[Density_bins_High],0.938871,2.819599


# Apriory

The second algorithm chosen is Apriori, it is based on the apriory principle that once there is a frequent item in the dataset, 
all of his subgroups are frequent as well. for example, if [CitricAcid_bins_High, Density_bins_High] are frequent, then [CitricAcid_bins_High] and [Density_bins_High] are frequent as well.
The algorithm is designed backword recursively, from K size items group in the set to k+1 size.
on each step, if we don't meet the minSupport criteria, the set is chopped.

It is easy to program but has a problem with big datasets, its time and space complexity are very high.

In [8]:
# The Apriori algorithm
# Credit: https://adataanalyst.com/machine-learning/apriori-algorithm-python-3-0/

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))#use frozen set so we
                            #can use it as a key in a dict    

def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData
    
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList  

In [9]:
data_apriori_ready = [data_dummies.columns[data_dummies.iloc[i].values==1].to_list() for i in data.index]

L,suppData = apriori(data_apriori_ready, minSupport=0.1)

In [10]:
suppData

{frozenset({'Alcohol_bins_Low'}): 0.2823878069432684,
 frozenset({'Chlorides_bins_Mid'}): 0.31922099915325997,
 frozenset({'CitricAcid_bins_High'}): 0.3323454699407282,
 frozenset({'Density_bins_High'}): 0.3329805249788315,
 frozenset({'FixedAcidity_bins_Mid'}): 0.3063082133784928,
 frozenset({'FreeSulfurDioxide_bins_High'}): 0.3141405588484335,
 frozenset({'Quality_bins_Mid'}): 0.4494072819644369,
 frozenset({'ResidualSugar_bins_High'}): 0.2483065198983912,
 frozenset({'Sulphates_bins_LowMid'}): 0.23116003386960204,
 frozenset({'TotalSulfurDioxide_bins_High'}): 0.331922099915326,
 frozenset({'VolatileAcidity_bins_MidHigh'}): 0.2449195596951736,
 frozenset({'pH_bins_Low'}): 0.3389077053344623,
 frozenset({'Chlorides_bins_High'}): 0.3196443691786621,
 frozenset({'CitricAcid_bins_Mid'}): 0.31837425910245554,
 frozenset({'Density_bins_Mid'}): 0.3329805249788315,
 frozenset({'FixedAcidity_bins_Low'}): 0.3689669771380186,
 frozenset({'FreeSulfurDioxide_bins_Low'}): 0.35076206604572396,
 fro

In [11]:
generateRules(L,suppData, minConf=0.8)

# Remark: the lift results for thos rules are the same.

frozenset({'Alcohol_bins_High'}) --> frozenset({'Density_bins_Low'}) conf: 0.8425760286225402
frozenset({'ResidualSugar_bins_High'}) --> frozenset({'Density_bins_High'}) conf: 0.8985507246376812


[(frozenset({'Alcohol_bins_High'}),
  frozenset({'Density_bins_Low'}),
  0.8425760286225402),
 (frozenset({'ResidualSugar_bins_High'}),
  frozenset({'Density_bins_High'}),
  0.8985507246376812)]

In [12]:
# Test:
# we want to see that the rules created by the Apriory algorithm can be found in the previous FP-Growth.

print(FP_rules.iloc[49])
print(FP_rules.iloc[75])

# Test passed.

antecedent    [ResidualSugar_bins_High]
consequent          [Density_bins_High]
confidence                     0.898551
lift                            2.69851
Name: 49, dtype: object
antecedent    [Alcohol_bins_High]
consequent     [Density_bins_Low]
confidence               0.842576
lift                      2.52239
Name: 75, dtype: object


# Compering the algorithms:
We can see both algorithms produced the same rules, the Apriori implementation gave us less rules, but it can be altered in order to give all the upper level frequents.
On the results found, the lift was similar, as expected.
We can see that the Apriory is focused on finding the items sets, while the FP-Growth give us straight forword the rules without further calculation needed in the apriori.
The FP-growh is also aranged is a tree structure, which help us to better understand the structure of the data.